In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!ls /content/drive/My\ Drive/DreamDB/dataset_tools

In [ ]:
!cat /content/drive/My\ Drive/DreamDB/dataset_tools

In [ ]:
import sys
sys.path.append('/content/drive/My Drive/DreamDB/')

In [ ]:
%load_ext autoreload
%autoreload 2
import dataset_tools as dataset

In [ ]:
%matplotlib inline
import os
import glob
import numpy as np
import pandas as pd
import math

In [ ]:
user = '76'

In [ ]:
url_data_loc = '/content/drive/MyDrive/DreamDB/User ' + user
url_save_loc = '/content/drive/MyDrive/DreamDB/Preprocess/User ' + user + '_processed.csv'

In [ ]:
tasks = []
for root, dirs, files in os.walk(url_data_loc):
    for f in files:
        if not f.endswith('.json'): continue
        data = dataset.open(os.path.join(root,f))
        t = data["task"]
        t['intervention'] = f.split('_')[2]
        t['sourceFile'] = f
        tasks.append(t)
sessions = pd.DataFrame(tasks)
sessions.sort_values(by='index',inplace=True)
sessions.set_index('index')

In [ ]:
user_files = []

files = glob.glob(url_data_loc + '/' + '*nter*')
for f in files:
    user_files.append(f)

print('No. of files ' + '(User ' + user + ') : ' + str(len(user_files)))

In [ ]:
li_interventions = []

for f in user_files:
  data = dataset.open(f)
  df = data.to_dataFrame()
  li_interventions.append(df)

df_interventions = pd.concat(li_interventions, axis = 0, ignore_index = True)

In [ ]:
task_indices = df_interventions.task_index.unique()

In [ ]:
print('Number of task indices: ' + str(len(task_indices)))

In [ ]:
print('List of dataframe columns: ')
df_interventions.columns

In [ ]:
li_tasks = []

for t in task_indices:
  df_temp = df_interventions.copy()
  df_tasks = df_temp.loc[df_temp['task_index'] == t]
  li_tasks.append(df_tasks)

print('Number of dataframes in task list: ' + str(len(li_tasks)))

In [ ]:
def multiDimenDist(point1,point2):
   deltaVals = [point2[dimension]-point1[dimension] for dimension in range(len(point1))]
   runningSquared = 0
   for coOrd in deltaVals:
       runningSquared += coOrd**2
   return runningSquared**(1/2)

def findVec(point1,point2,unitSphere = False):
  finalVector = [0 for coOrd in point1]
  for dimension, coOrd in enumerate(point1):
      deltaCoOrd = point2[dimension]-coOrd
      finalVector[dimension] = deltaCoOrd
  if unitSphere:
      totalDist = multiDimenDist(point1,point2)
      unitVector =[]
      for dimen in finalVector:
          unitVector.append( dimen/totalDist)
      return unitVector
  else:
      return finalVector

In [ ]:
def findAngle(vector_1, vector_2):
  unit_vector_1 = vector_1 / np.linalg.norm(vector_1)
  unit_vector_2 = vector_2 / np.linalg.norm(vector_2)
  dot_product = np.dot(unit_vector_1, unit_vector_2)
  rad = np.arccos(dot_product)
  deg = np.rad2deg(rad)
  return deg

In [ ]:
def FindCapEH(erx,ery,erz):
  CapE=math.sqrt(erx**2+ery**2+erz**2)
  return CapE
def FindAlpha(erx,ery,erz):
  alpha=math.degrees(math.acos(erx/FindCapEH(erx,ery,erz)))
  return alpha
def FindBeta(erx,ery,erz):
  beta=math.degrees(math.acos(ery/FindCapEH(erx,ery,erz)))
  return beta
def FindGama(erx,ery,erz):
  gama=math.degrees(math.acos(erz/FindCapEH(erx,ery,erz)))
  return gama

In [ ]:
def Theta(eye,head):
  theta=math.degrees(math.acos((np.dot(eye,head)/(FindCapEH(eye[0], eye[1], eye[2]) * FindCapEH(head[0], head[1], head[2])))))
  return theta
def Phi(eye,head):
  H = FindCapEH(head[0], head[1], head[2])
  E = FindCapEH(eye[0], eye[1], eye[2])
  theta = Theta(eye, head)
  phi = math.degrees(math.atan((H * (math.degrees(math.sin(theta)))) / (E + (H * (math.degrees(math.cos(theta)))))))
  return phi

In [ ]:
def getDist(p1, p2, l):
  li_dist = []
  for i in range(l):
    #d = math.sqrt(((p1[i][0] - p2[i][0])**2) + ((p1[i][1] - p2[i][1])**2) + ((p1[i][2] - p2[i][2])**2))
    d = multiDimenDist(p1[i], p2[i])
    li_dist.append(d)
  arr_dist = np.array(li_dist)
  mean_dist = np.mean(arr_dist)
  return mean_dist

In [ ]:
def getAngle(p1, p2, p3, l):
  li_angle = []
  for i in range(l):
    p2p1 = findVec(p2[i], p1[i])
    p2p3 = findVec(p2[i], p3[i])
    a = findAngle(p2p1, p2p3)
    li_angle.append(a)
  arr_angle = np.array(li_angle)
  mean_angle = np.mean(arr_angle)
  return mean_angle

In [ ]:
def getAlpha(rx, ry, rz, l):
  li_a = []
  for i in range(l):
    a = FindAlpha(rx[i], ry[i], rz[i])
    li_a.append(a)
  arr_a = np.array(li_a)
  mean_a = np.mean(arr_a)
  return mean_a

def getBeta(rx, ry, rz, l):
  li_b = []
  for i in range(l):
    b = FindBeta(rx[i], ry[i], rz[i])
    li_b.append(b)
  arr_b = np.array(li_b)
  mean_b = np.mean(arr_b)
  return mean_b

def getGamma(rx, ry, rz, l):
  li_g = []
  for i in range(l):
    g = FindGama(rx[i], ry[i], rz[i])
    li_g.append(g)
  arr_g = np.array(li_g)
  mean_g = np.mean(arr_g)
  return mean_g

def getTheta(eye, head, l):
  li_t = []
  for i in range(l):
    t = Theta(eye[i], head[i])
    li_t.append(t)
  arr_t = np.array(li_t)
  mean_t = np.mean(arr_t)
  return mean_t

def getPhi(eye, head, l):
  li_p = []
  for i in range(l):
    p = Phi(eye[i], head[i])
    li_p.append(p)
  arr_p = np.array(li_p)
  mean_p = np.mean(arr_p)
  return mean_p

In [ ]:
from itertools import combinations

comb = combinations(['H', 'SL', 'SC', 'SR', 'EL', 'ER', 'HL', 'HR', 'WL', 'WR'], 2)
comb = list(comb)
print(len(comb))
for i in comb:
    print(i)

In [ ]:
#Joints (10): H, SL, SC, SR, EL, ER, HL, HR, WL, WR
#Number of Features: 103

li_processed = []

col_list = ['H_SL_dist', 'H_SC_dist', 'H_SR_dist', 'H_EL_dist', 'H_ER_dist', 'H_HL_dist', 'H_HR_dist', 'H_WL_dist', 'H_WR_dist', 'SL_SC_dist',
            'SL_SR_dist', 'SL_EL_dist', 'SL_ER_dist', 'SL_HL_dist', 'SL_HR_dist', 'SL_WL_dist', 'SL_WR_dist', 'SC_SR_dist', 'SC_EL_dist', 'SC_ER_dist',
            'SC_HL_dist', 'SC_HR_dist', 'SC_WL_dist', 'SC_WR_dist', 'SR_EL_dist', 'SR_ER_dist', 'SR_HL_dist', 'SR_HR_dist', 'SR_WL_dist', 'SR_WR_dist',
            'EL_ER_dist', 'EL_HL_dist', 'EL_HR_dist', 'EL_WL_dist', 'EL_WR_dist', 'ER_HL_dist', 'ER_HR_dist', 'ER_WL_dist', 'ER_WR_dist', 'HL_HR_dist',
            'HL_WL_dist', 'HL_WR_dist', 'HR_WL_dist', 'HR_WR_dist', 'WL_WR_dist',
            'H_SC_SL_angle', 'H_SC_SR_angle', 'SL_SC_SR_angle',
            'SC_SL_EL_angle', 'SC_SR_ER_angle',
            'SL_EL_HL_angle', 'SR_ER_HR_angle',
            'EL_HL_WL_angle', 'ER_HR_WR_angle',
            'alpha_e', 'beta_e', 'gamma_e', 'alpha_h', 'beta_h', 'gamma_h', 'theta', 'phi',
            'skeleton_head_x_std', 'skeleton_head_y_std', 'skeleton_head_z_std',
            'skeleton_sholder_center_x_std', 'skeleton_sholder_center_y_std', 'skeleton_sholder_center_z_std',
            'skeleton_sholder_left_x_std', 'skeleton_sholder_left_y_std', 'skeleton_sholder_left_z_std',
            'skeleton_sholder_right_x_std', 'skeleton_sholder_right_y_std', 'skeleton_sholder_right_z_std',
            'skeleton_elbow_left_x_std', 'skeleton_elbow_left_y_std', 'skeleton_elbow_left_z_std',
            'skeleton_elbow_right_x_std', 'skeleton_elbow_right_y_std', 'skeleton_elbow_right_z_std',
            'skeleton_hand_left_x_std', 'skeleton_hand_left_y_std', 'skeleton_hand_left_z_std',
            'skeleton_hand_right_x_std', 'skeleton_hand_right_y_std', 'skeleton_hand_right_z_std',
            'skeleton_wrist_left_x_std', 'skeleton_wrist_left_y_std', 'skeleton_wrist_left_z_std',
            'skeleton_wrist_right_x_std', 'skeleton_wrist_right_y_std', 'skeleton_wrist_right_z_std',
            'eye_gaze_rx_std', 'eye_gaze_ry_std', 'eye_gaze_rz_std',
            'head_gaze_rx_std', 'head_gaze_ry_std', 'head_gaze_rz_std',
            'participant_ageInMonths', 'age_group', 'participant_gender',
            'task_ability', 'task_difficultyLevel',
            'ados_preTest_communication', 'ados_preTest_interaction', 'ados_preTest_module', 'ados_preTest_play', 'ados_preTest_protocol',
            'ados_preTest_socialCommunicationQuestionnaire', 'ados_preTest_stereotype', 'ados_preTest_total', 'condition', 'level']

col_drop_list = ['frame_rate', 'participant_id', 'skeleton_elbow_left_confidence', 'skeleton_elbow_right_confidence', 'skeleton_hand_left_confidence',
                 'skeleton_hand_right_confidence', 'skeleton_head_confidence', 'skeleton_sholder_center_confidence', 'skeleton_sholder_left_confidence',
                 'skeleton_sholder_right_confidence', 'skeleton_wrist_left_confidence', 'skeleton_wrist_right_confidence',
                 'task_end', 'task_index', 'task_start', 'time']

for i in range(len(li_tasks)):
  df_task_temp = li_tasks[i].copy()
  df_task_temp = df_task_temp.drop(columns = col_drop_list)
  df_processed = pd.DataFrame(columns = col_list)

  for j in range(0, len(df_task_temp), 25):
    k = j + 25
    if (k > len(df_task_temp)):
      k = len(df_task_temp)

    temp = df_task_temp.iloc[j : k].copy()
    temp = temp.dropna()
    if temp.empty:
      continue
    li_temp = []

    #Joint Distance Features (45)
    H_SL_dist = getDist(temp[['skeleton_head_x', 'skeleton_head_y', 'skeleton_head_z']].to_numpy(),
                        temp[['skeleton_sholder_left_x', 'skeleton_sholder_left_y', 'skeleton_sholder_left_z']].to_numpy(),
                        len(temp[['skeleton_head_x', 'skeleton_head_y', 'skeleton_head_z']].to_numpy()))
    li_temp.append(H_SL_dist)
    H_SC_dist = getDist(temp[['skeleton_head_x', 'skeleton_head_y', 'skeleton_head_z']].to_numpy(),
                        temp[['skeleton_sholder_center_x', 'skeleton_sholder_center_y', 'skeleton_sholder_center_z']].to_numpy(),
                        len(temp[['skeleton_head_x', 'skeleton_head_y', 'skeleton_head_z']].to_numpy()))
    li_temp.append(H_SC_dist)
    H_SR_dist = getDist(temp[['skeleton_head_x', 'skeleton_head_y', 'skeleton_head_z']].to_numpy(),
                        temp[['skeleton_sholder_right_x', 'skeleton_sholder_right_y', 'skeleton_sholder_right_z']].to_numpy(),
                        len(temp[['skeleton_head_x', 'skeleton_head_y', 'skeleton_head_z']].to_numpy()))
    li_temp.append(H_SR_dist)
    H_EL_dist = getDist(temp[['skeleton_head_x', 'skeleton_head_y', 'skeleton_head_z']].to_numpy(),
                        temp[['skeleton_elbow_left_x', 'skeleton_elbow_left_y', 'skeleton_elbow_left_z']].to_numpy(),
                        len(temp[['skeleton_head_x', 'skeleton_head_y', 'skeleton_head_z']].to_numpy()))
    li_temp.append(H_EL_dist)
    H_ER_dist = getDist(temp[['skeleton_head_x', 'skeleton_head_y', 'skeleton_head_z']].to_numpy(),
                        temp[['skeleton_elbow_right_x', 'skeleton_elbow_right_y', 'skeleton_elbow_right_z']].to_numpy(),
                        len(temp[['skeleton_head_x', 'skeleton_head_y', 'skeleton_head_z']].to_numpy()))
    li_temp.append(H_ER_dist)
    H_HL_dist = getDist(temp[['skeleton_head_x', 'skeleton_head_y', 'skeleton_head_z']].to_numpy(),
                        temp[['skeleton_hand_left_x', 'skeleton_hand_left_y', 'skeleton_hand_left_z']].to_numpy(),
                        len(temp[['skeleton_head_x', 'skeleton_head_y', 'skeleton_head_z']].to_numpy()))
    li_temp.append(H_HL_dist)
    H_HR_dist = getDist(temp[['skeleton_head_x', 'skeleton_head_y', 'skeleton_head_z']].to_numpy(),
                        temp[['skeleton_hand_right_x', 'skeleton_hand_right_y', 'skeleton_hand_right_z']].to_numpy(),
                        len(temp[['skeleton_head_x', 'skeleton_head_y', 'skeleton_head_z']].to_numpy()))
    li_temp.append(H_HR_dist)
    H_WL_dist = getDist(temp[['skeleton_head_x', 'skeleton_head_y', 'skeleton_head_z']].to_numpy(),
                        temp[['skeleton_wrist_left_x', 'skeleton_wrist_left_y', 'skeleton_wrist_left_z']].to_numpy(),
                        len(temp[['skeleton_head_x', 'skeleton_head_y', 'skeleton_head_z']].to_numpy()))
    li_temp.append(H_WL_dist)
    H_WR_dist = getDist(temp[['skeleton_head_x', 'skeleton_head_y', 'skeleton_head_z']].to_numpy(),
                        temp[['skeleton_wrist_right_x', 'skeleton_wrist_right_y', 'skeleton_wrist_right_z']].to_numpy(),
                        len(temp[['skeleton_head_x', 'skeleton_head_y', 'skeleton_head_z']].to_numpy()))
    li_temp.append(H_WR_dist)
    SL_SC_dist = getDist(temp[['skeleton_sholder_left_x', 'skeleton_sholder_left_y', 'skeleton_sholder_left_z']].to_numpy(),
                         temp[['skeleton_sholder_center_x', 'skeleton_sholder_center_y', 'skeleton_sholder_center_z']].to_numpy(),
                         len(temp[['skeleton_sholder_left_x', 'skeleton_sholder_left_y', 'skeleton_sholder_left_z']].to_numpy()))
    li_temp.append(SL_SC_dist)
    SL_SR_dist = getDist(temp[['skeleton_sholder_left_x', 'skeleton_sholder_left_y', 'skeleton_sholder_left_z']].to_numpy(),
                         temp[['skeleton_sholder_right_x', 'skeleton_sholder_right_y', 'skeleton_sholder_right_z']].to_numpy(),
                         len(temp[['skeleton_sholder_left_x', 'skeleton_sholder_left_y', 'skeleton_sholder_left_z']].to_numpy()))
    li_temp.append(SL_SR_dist)
    SL_EL_dist = getDist(temp[['skeleton_sholder_left_x', 'skeleton_sholder_left_y', 'skeleton_sholder_left_z']].to_numpy(),
                         temp[['skeleton_elbow_left_x', 'skeleton_elbow_left_y', 'skeleton_elbow_left_z']].to_numpy(),
                         len(temp[['skeleton_sholder_left_x', 'skeleton_sholder_left_y', 'skeleton_sholder_left_z']].to_numpy()))
    li_temp.append(SL_EL_dist)
    SL_ER_dist = getDist(temp[['skeleton_sholder_left_x', 'skeleton_sholder_left_y', 'skeleton_sholder_left_z']].to_numpy(),
                         temp[['skeleton_elbow_right_x', 'skeleton_elbow_right_y', 'skeleton_elbow_right_z']].to_numpy(),
                         len(temp[['skeleton_sholder_left_x', 'skeleton_sholder_left_y', 'skeleton_sholder_left_z']].to_numpy()))
    li_temp.append(SL_ER_dist)
    SL_HL_dist = getDist(temp[['skeleton_sholder_left_x', 'skeleton_sholder_left_y', 'skeleton_sholder_left_z']].to_numpy(),
                         temp[['skeleton_hand_left_x', 'skeleton_hand_left_y', 'skeleton_hand_left_z']].to_numpy(),
                         len(temp[['skeleton_sholder_left_x', 'skeleton_sholder_left_y', 'skeleton_sholder_left_z']].to_numpy()))
    li_temp.append(SL_HL_dist)
    SL_HR_dist = getDist(temp[['skeleton_sholder_left_x', 'skeleton_sholder_left_y', 'skeleton_sholder_left_z']].to_numpy(),
                         temp[['skeleton_hand_right_x', 'skeleton_hand_right_y', 'skeleton_hand_right_z']].to_numpy(),
                         len(temp[['skeleton_sholder_left_x', 'skeleton_sholder_left_y', 'skeleton_sholder_left_z']].to_numpy()))
    li_temp.append(SL_HR_dist)
    SL_WL_dist = getDist(temp[['skeleton_sholder_left_x', 'skeleton_sholder_left_y', 'skeleton_sholder_left_z']].to_numpy(),
                         temp[['skeleton_wrist_left_x', 'skeleton_wrist_left_y', 'skeleton_wrist_left_z']].to_numpy(),
                         len(temp[['skeleton_sholder_left_x', 'skeleton_sholder_left_y', 'skeleton_sholder_left_z']].to_numpy()))
    li_temp.append(SL_WL_dist)
    SL_WR_dist = getDist(temp[['skeleton_sholder_left_x', 'skeleton_sholder_left_y', 'skeleton_sholder_left_z']].to_numpy(),
                         temp[['skeleton_wrist_right_x', 'skeleton_wrist_right_y', 'skeleton_wrist_right_z']].to_numpy(),
                         len(temp[['skeleton_sholder_left_x', 'skeleton_sholder_left_y', 'skeleton_sholder_left_z']].to_numpy()))
    li_temp.append(SL_WR_dist)
    SC_SR_dist = getDist(temp[['skeleton_sholder_center_x', 'skeleton_sholder_center_y', 'skeleton_sholder_center_z']].to_numpy(),
                         temp[['skeleton_sholder_right_x', 'skeleton_sholder_right_y', 'skeleton_sholder_right_z']].to_numpy(),
                         len(temp[['skeleton_sholder_center_x', 'skeleton_sholder_center_y', 'skeleton_sholder_center_z']].to_numpy()))
    li_temp.append(SC_SR_dist)
    SC_EL_dist = getDist(temp[['skeleton_sholder_center_x', 'skeleton_sholder_center_y', 'skeleton_sholder_center_z']].to_numpy(),
                         temp[['skeleton_elbow_left_x', 'skeleton_elbow_left_y', 'skeleton_elbow_left_z']].to_numpy(),
                         len(temp[['skeleton_sholder_center_x', 'skeleton_sholder_center_y', 'skeleton_sholder_center_z']].to_numpy()))
    li_temp.append(SC_EL_dist)
    SC_ER_dist = getDist(temp[['skeleton_sholder_center_x', 'skeleton_sholder_center_y', 'skeleton_sholder_center_z']].to_numpy(),
                         temp[['skeleton_elbow_right_x', 'skeleton_elbow_right_y', 'skeleton_elbow_right_z']].to_numpy(),
                         len(temp[['skeleton_sholder_center_x', 'skeleton_sholder_center_y', 'skeleton_sholder_center_z']].to_numpy()))
    li_temp.append(SC_ER_dist)
    SC_HL_dist = getDist(temp[['skeleton_sholder_center_x', 'skeleton_sholder_center_y', 'skeleton_sholder_center_z']].to_numpy(),
                         temp[['skeleton_hand_left_x', 'skeleton_hand_left_y', 'skeleton_hand_left_z']].to_numpy(),
                         len(temp[['skeleton_sholder_center_x', 'skeleton_sholder_center_y', 'skeleton_sholder_center_z']].to_numpy()))
    li_temp.append(SC_HL_dist)
    SC_HR_dist = getDist(temp[['skeleton_sholder_center_x', 'skeleton_sholder_center_y', 'skeleton_sholder_center_z']].to_numpy(),
                         temp[['skeleton_hand_right_x', 'skeleton_hand_right_y', 'skeleton_hand_right_z']].to_numpy(),
                         len(temp[['skeleton_sholder_center_x', 'skeleton_sholder_center_y', 'skeleton_sholder_center_z']].to_numpy()))
    li_temp.append(SC_HR_dist)
    SC_WL_dist = getDist(temp[['skeleton_sholder_center_x', 'skeleton_sholder_center_y', 'skeleton_sholder_center_z']].to_numpy(),
                         temp[['skeleton_wrist_left_x', 'skeleton_wrist_left_y', 'skeleton_wrist_left_z']].to_numpy(),
                         len(temp[['skeleton_sholder_center_x', 'skeleton_sholder_center_y', 'skeleton_sholder_center_z']].to_numpy()))
    li_temp.append(SC_WL_dist)
    SC_WR_dist = getDist(temp[['skeleton_sholder_center_x', 'skeleton_sholder_center_y', 'skeleton_sholder_center_z']].to_numpy(),
                         temp[['skeleton_wrist_right_x', 'skeleton_wrist_right_y', 'skeleton_wrist_right_z']].to_numpy(),
                         len(temp[['skeleton_sholder_center_x', 'skeleton_sholder_center_y', 'skeleton_sholder_center_z']].to_numpy()))
    li_temp.append(SC_WR_dist)
    SR_EL_dist = getDist(temp[['skeleton_sholder_right_x', 'skeleton_sholder_right_y', 'skeleton_sholder_right_z']].to_numpy(),
                         temp[['skeleton_elbow_left_x', 'skeleton_elbow_left_y', 'skeleton_elbow_left_z']].to_numpy(),
                         len(temp[['skeleton_sholder_right_x', 'skeleton_sholder_right_y', 'skeleton_sholder_right_z']].to_numpy()))
    li_temp.append(SR_EL_dist)
    SR_ER_dist = getDist(temp[['skeleton_sholder_right_x', 'skeleton_sholder_right_y', 'skeleton_sholder_right_z']].to_numpy(),
                         temp[['skeleton_elbow_right_x', 'skeleton_elbow_right_y', 'skeleton_elbow_right_z']].to_numpy(),
                         len(temp[['skeleton_sholder_right_x', 'skeleton_sholder_right_y', 'skeleton_sholder_right_z']].to_numpy()))
    li_temp.append(SR_ER_dist)
    SR_HL_dist = getDist(temp[['skeleton_sholder_right_x', 'skeleton_sholder_right_y', 'skeleton_sholder_right_z']].to_numpy(),
                         temp[['skeleton_hand_left_x', 'skeleton_hand_left_y', 'skeleton_hand_left_z']].to_numpy(),
                         len(temp[['skeleton_sholder_right_x', 'skeleton_sholder_right_y', 'skeleton_sholder_right_z']].to_numpy()))
    li_temp.append(SR_HL_dist)
    SR_HR_dist = getDist(temp[['skeleton_sholder_right_x', 'skeleton_sholder_right_y', 'skeleton_sholder_right_z']].to_numpy(),
                         temp[['skeleton_hand_right_x', 'skeleton_hand_right_y', 'skeleton_hand_right_z']].to_numpy(),
                         len(temp[['skeleton_sholder_right_x', 'skeleton_sholder_right_y', 'skeleton_sholder_right_z']].to_numpy()))
    li_temp.append(SR_HR_dist)
    SR_WL_dist = getDist(temp[['skeleton_sholder_right_x', 'skeleton_sholder_right_y', 'skeleton_sholder_right_z']].to_numpy(),
                         temp[['skeleton_wrist_left_x', 'skeleton_wrist_left_y', 'skeleton_wrist_left_z']].to_numpy(),
                         len(temp[['skeleton_sholder_right_x', 'skeleton_sholder_right_y', 'skeleton_sholder_right_z']].to_numpy()))
    li_temp.append(SR_WL_dist)
    SR_WR_dist = getDist(temp[['skeleton_sholder_right_x', 'skeleton_sholder_right_y', 'skeleton_sholder_right_z']].to_numpy(),
                         temp[['skeleton_wrist_right_x', 'skeleton_wrist_right_y', 'skeleton_wrist_right_z']].to_numpy(),
                         len(temp[['skeleton_sholder_right_x', 'skeleton_sholder_right_y', 'skeleton_sholder_right_z']].to_numpy()))
    li_temp.append(SR_WR_dist)
    EL_ER_dist = getDist(temp[['skeleton_elbow_left_x', 'skeleton_elbow_left_y', 'skeleton_elbow_left_z']].to_numpy(),
                         temp[['skeleton_elbow_right_x', 'skeleton_elbow_right_y', 'skeleton_elbow_right_z']].to_numpy(),
                         len(temp[['skeleton_elbow_left_x', 'skeleton_elbow_left_y', 'skeleton_elbow_left_z']].to_numpy()))
    li_temp.append(EL_ER_dist)
    EL_HL_dist = getDist(temp[['skeleton_elbow_left_x', 'skeleton_elbow_left_y', 'skeleton_elbow_left_z']].to_numpy(),
                         temp[['skeleton_hand_left_x', 'skeleton_hand_left_y', 'skeleton_hand_left_z']].to_numpy(),
                         len(temp[['skeleton_elbow_left_x', 'skeleton_elbow_left_y', 'skeleton_elbow_left_z']].to_numpy()))
    li_temp.append(EL_HL_dist)
    EL_HR_dist = getDist(temp[['skeleton_elbow_left_x', 'skeleton_elbow_left_y', 'skeleton_elbow_left_z']].to_numpy(),
                         temp[['skeleton_hand_right_x', 'skeleton_hand_right_y', 'skeleton_hand_right_z']].to_numpy(),
                         len(temp[['skeleton_elbow_left_x', 'skeleton_elbow_left_y', 'skeleton_elbow_left_z']].to_numpy()))
    li_temp.append(EL_HR_dist)
    EL_WL_dist = getDist(temp[['skeleton_elbow_left_x', 'skeleton_elbow_left_y', 'skeleton_elbow_left_z']].to_numpy(),
                         temp[['skeleton_wrist_left_x', 'skeleton_wrist_left_y', 'skeleton_wrist_left_z']].to_numpy(),
                         len(temp[['skeleton_elbow_left_x', 'skeleton_elbow_left_y', 'skeleton_elbow_left_z']].to_numpy()))
    li_temp.append(EL_WL_dist)
    EL_WR_dist = getDist(temp[['skeleton_elbow_left_x', 'skeleton_elbow_left_y', 'skeleton_elbow_left_z']].to_numpy(),
                         temp[['skeleton_wrist_right_x', 'skeleton_wrist_right_y', 'skeleton_wrist_right_z']].to_numpy(),
                         len(temp[['skeleton_elbow_left_x', 'skeleton_elbow_left_y', 'skeleton_elbow_left_z']].to_numpy()))
    li_temp.append(EL_WR_dist)
    ER_HL_dist = getDist(temp[['skeleton_elbow_right_x', 'skeleton_elbow_right_y', 'skeleton_elbow_right_z']].to_numpy(),
                         temp[['skeleton_hand_left_x', 'skeleton_hand_left_y', 'skeleton_hand_left_z']].to_numpy(),
                         len(temp[['skeleton_elbow_right_x', 'skeleton_elbow_right_y', 'skeleton_elbow_right_z']].to_numpy()))
    li_temp.append(ER_HL_dist)
    ER_HR_dist = getDist(temp[['skeleton_elbow_right_x', 'skeleton_elbow_right_y', 'skeleton_elbow_right_z']].to_numpy(),
                         temp[['skeleton_hand_right_x', 'skeleton_hand_right_y', 'skeleton_hand_right_z']].to_numpy(),
                         len(temp[['skeleton_elbow_right_x', 'skeleton_elbow_right_y', 'skeleton_elbow_right_z']].to_numpy()))
    li_temp.append(ER_HR_dist)
    ER_WL_dist = getDist(temp[['skeleton_elbow_right_x', 'skeleton_elbow_right_y', 'skeleton_elbow_right_z']].to_numpy(),
                         temp[['skeleton_wrist_left_x', 'skeleton_wrist_left_y', 'skeleton_wrist_left_z']].to_numpy(),
                         len(temp[['skeleton_elbow_right_x', 'skeleton_elbow_right_y', 'skeleton_elbow_right_z']].to_numpy()))
    li_temp.append(ER_WL_dist)
    ER_WR_dist = getDist(temp[['skeleton_elbow_right_x', 'skeleton_elbow_right_y', 'skeleton_elbow_right_z']].to_numpy(),
                         temp[['skeleton_wrist_right_x', 'skeleton_wrist_right_y', 'skeleton_wrist_right_z']].to_numpy(),
                         len(temp[['skeleton_elbow_right_x', 'skeleton_elbow_right_y', 'skeleton_elbow_right_z']].to_numpy()))
    li_temp.append(ER_WR_dist)
    HL_HR_dist = getDist(temp[['skeleton_hand_left_x', 'skeleton_hand_left_y', 'skeleton_hand_left_z']].to_numpy(),
                         temp[['skeleton_hand_right_x', 'skeleton_hand_right_y', 'skeleton_hand_right_z']].to_numpy(),
                         len(temp[['skeleton_hand_left_x', 'skeleton_hand_left_y', 'skeleton_hand_left_z']].to_numpy()))
    li_temp.append(HL_HR_dist)
    HL_WL_dist = getDist(temp[['skeleton_hand_left_x', 'skeleton_hand_left_y', 'skeleton_hand_left_z']].to_numpy(),
                         temp[['skeleton_wrist_left_x', 'skeleton_wrist_left_y', 'skeleton_wrist_left_z']].to_numpy(),
                         len(temp[['skeleton_hand_left_x', 'skeleton_hand_left_y', 'skeleton_hand_left_z']].to_numpy()))
    li_temp.append(HL_WL_dist)
    HL_WR_dist = getDist(temp[['skeleton_hand_left_x', 'skeleton_hand_left_y', 'skeleton_hand_left_z']].to_numpy(),
                         temp[['skeleton_wrist_right_x', 'skeleton_wrist_right_y', 'skeleton_wrist_right_z']].to_numpy(),
                         len(temp[['skeleton_hand_left_x', 'skeleton_hand_left_y', 'skeleton_hand_left_z']].to_numpy()))
    li_temp.append(HL_WR_dist)
    HR_WL_dist = getDist(temp[['skeleton_hand_right_x', 'skeleton_hand_right_y', 'skeleton_hand_right_z']].to_numpy(),
                         temp[['skeleton_wrist_left_x', 'skeleton_wrist_left_y', 'skeleton_wrist_left_z']].to_numpy(),
                         len(temp[['skeleton_hand_right_x', 'skeleton_hand_right_y', 'skeleton_hand_right_z']].to_numpy()))
    li_temp.append(HR_WL_dist)
    HR_WR_dist = getDist(temp[['skeleton_hand_right_x', 'skeleton_hand_right_y', 'skeleton_hand_right_z']].to_numpy(),
                         temp[['skeleton_wrist_right_x', 'skeleton_wrist_right_y', 'skeleton_wrist_right_z']].to_numpy(),
                         len(temp[['skeleton_hand_right_x', 'skeleton_hand_right_y', 'skeleton_hand_right_z']].to_numpy()))
    li_temp.append(HR_WR_dist)
    WL_WR_dist = getDist(temp[['skeleton_wrist_left_x', 'skeleton_wrist_left_y', 'skeleton_wrist_left_z']].to_numpy(),
                         temp[['skeleton_wrist_right_x', 'skeleton_wrist_right_y', 'skeleton_wrist_right_z']].to_numpy(),
                         len(temp[['skeleton_wrist_left_x', 'skeleton_wrist_left_y', 'skeleton_wrist_left_z']].to_numpy()))
    li_temp.append(WL_WR_dist)

    #Joint Angle Features (9)
    H_SC_SL_angle = getAngle(temp[['skeleton_head_x', 'skeleton_head_y', 'skeleton_head_z']].to_numpy(),
                             temp[['skeleton_sholder_center_x', 'skeleton_sholder_center_y', 'skeleton_sholder_center_z']].to_numpy(),
                             temp[['skeleton_sholder_left_x', 'skeleton_sholder_left_y', 'skeleton_sholder_left_z']].to_numpy(),
                             len(temp[['skeleton_sholder_center_x', 'skeleton_sholder_center_y', 'skeleton_sholder_center_z']].to_numpy()))
    li_temp.append(H_SC_SL_angle)
    H_SC_SR_angle = getAngle(temp[['skeleton_head_x', 'skeleton_head_y', 'skeleton_head_z']].to_numpy(),
                             temp[['skeleton_sholder_center_x', 'skeleton_sholder_center_y', 'skeleton_sholder_center_z']].to_numpy(),
                             temp[['skeleton_sholder_right_x', 'skeleton_sholder_right_y', 'skeleton_sholder_right_z']].to_numpy(),
                             len(temp[['skeleton_sholder_center_x', 'skeleton_sholder_center_y', 'skeleton_sholder_center_z']].to_numpy()))
    li_temp.append(H_SC_SR_angle)
    SL_SC_SR_angle = getAngle(temp[['skeleton_sholder_left_x', 'skeleton_sholder_left_y', 'skeleton_sholder_left_z']].to_numpy(),
                              temp[['skeleton_sholder_center_x', 'skeleton_sholder_center_y', 'skeleton_sholder_center_z']].to_numpy(),
                              temp[['skeleton_sholder_right_x', 'skeleton_sholder_right_y', 'skeleton_sholder_right_z']].to_numpy(),
                              len(temp[['skeleton_sholder_center_x', 'skeleton_sholder_center_y', 'skeleton_sholder_center_z']].to_numpy()))
    li_temp.append(SL_SC_SR_angle)
    SC_SL_EL_angle = getAngle(temp[['skeleton_sholder_center_x', 'skeleton_sholder_center_y', 'skeleton_sholder_center_z']].to_numpy(),
                              temp[['skeleton_sholder_left_x', 'skeleton_sholder_left_y', 'skeleton_sholder_left_z']].to_numpy(),
                              temp[['skeleton_elbow_left_x', 'skeleton_elbow_left_y', 'skeleton_elbow_left_z']].to_numpy(),
                              len(temp[['skeleton_sholder_left_x', 'skeleton_sholder_left_y', 'skeleton_sholder_left_z']].to_numpy()))
    li_temp.append(SC_SL_EL_angle)
    SC_SR_ER_angle = getAngle(temp[['skeleton_sholder_center_x', 'skeleton_sholder_center_y', 'skeleton_sholder_center_z']].to_numpy(),
                              temp[['skeleton_sholder_right_x', 'skeleton_sholder_right_y', 'skeleton_sholder_right_z']].to_numpy(),
                              temp[['skeleton_elbow_right_x', 'skeleton_elbow_right_y', 'skeleton_elbow_right_z']].to_numpy(),
                              len(temp[['skeleton_sholder_right_x', 'skeleton_sholder_right_y', 'skeleton_sholder_right_z']].to_numpy()))
    li_temp.append(SC_SR_ER_angle)
    SL_EL_HL_angle = getAngle(temp[['skeleton_sholder_left_x', 'skeleton_sholder_left_y', 'skeleton_sholder_left_z']].to_numpy(),
                              temp[['skeleton_elbow_left_x', 'skeleton_elbow_left_y', 'skeleton_elbow_left_z']].to_numpy(),
                              temp[['skeleton_hand_left_x', 'skeleton_hand_left_y', 'skeleton_hand_left_z']].to_numpy(),
                              len(temp[['skeleton_elbow_left_x', 'skeleton_elbow_left_y', 'skeleton_elbow_left_z']].to_numpy()))
    li_temp.append(SL_EL_HL_angle)
    SR_ER_HR_angle = getAngle(temp[['skeleton_sholder_right_x', 'skeleton_sholder_right_y', 'skeleton_sholder_right_z']].to_numpy(),
                              temp[['skeleton_elbow_right_x', 'skeleton_elbow_right_y', 'skeleton_elbow_right_z']].to_numpy(),
                              temp[['skeleton_hand_right_x', 'skeleton_hand_right_y', 'skeleton_hand_right_z']].to_numpy(),
                              len(temp[['skeleton_elbow_right_x', 'skeleton_elbow_right_y', 'skeleton_elbow_right_z']].to_numpy()))
    li_temp.append(SR_ER_HR_angle)
    EL_HL_WL_angle = getAngle(temp[['skeleton_elbow_left_x', 'skeleton_elbow_left_y', 'skeleton_elbow_left_z']].to_numpy(),
                              temp[['skeleton_hand_left_x', 'skeleton_hand_left_y', 'skeleton_hand_left_z']].to_numpy(),
                              temp[['skeleton_wrist_left_x', 'skeleton_wrist_left_y', 'skeleton_wrist_left_z']].to_numpy(),
                              len(temp[['skeleton_hand_left_x', 'skeleton_hand_left_y', 'skeleton_hand_left_z']].to_numpy()))
    li_temp.append(EL_HL_WL_angle)
    ER_HR_WR_angle = getAngle(temp[['skeleton_elbow_right_x', 'skeleton_elbow_right_y', 'skeleton_elbow_right_z']].to_numpy(),
                              temp[['skeleton_hand_right_x', 'skeleton_hand_right_y', 'skeleton_hand_right_z']].to_numpy(),
                              temp[['skeleton_wrist_right_x', 'skeleton_wrist_right_y', 'skeleton_wrist_right_z']].to_numpy(),
                              len(temp[['skeleton_hand_right_x', 'skeleton_hand_right_y', 'skeleton_hand_right_z']].to_numpy()))
    li_temp.append(ER_HR_WR_angle)

    #Gaze Vector Features (8)
    alpha_e = getAlpha(temp[['eye_gaze_rx']].to_numpy(),
                       temp[['eye_gaze_ry']].to_numpy(),
                       temp[['eye_gaze_rz']].to_numpy(),
                       len(temp[['eye_gaze_rx']].to_numpy()))
    li_temp.append(alpha_e)
    beta_e = getBeta(temp[['eye_gaze_rx']].to_numpy(),
                       temp[['eye_gaze_ry']].to_numpy(),
                       temp[['eye_gaze_rz']].to_numpy(),
                       len(temp[['eye_gaze_ry']].to_numpy()))
    li_temp.append(beta_e)
    gamma_e = getGamma(temp[['eye_gaze_rx']].to_numpy(),
                       temp[['eye_gaze_ry']].to_numpy(),
                       temp[['eye_gaze_rz']].to_numpy(),
                       len(temp[['eye_gaze_rz']].to_numpy()))
    li_temp.append(gamma_e)
    alpha_h = getAlpha(temp[['head_gaze_rx']].to_numpy(),
                       temp[['head_gaze_ry']].to_numpy(),
                       temp[['head_gaze_rz']].to_numpy(),
                       len(temp[['head_gaze_rx']].to_numpy()))
    li_temp.append(alpha_h)
    beta_h = getBeta(temp[['head_gaze_rx']].to_numpy(),
                     temp[['head_gaze_ry']].to_numpy(),
                     temp[['head_gaze_rz']].to_numpy(),
                     len(temp[['head_gaze_ry']].to_numpy()))
    li_temp.append(beta_h)
    gamma_h = getGamma(temp[['head_gaze_rx']].to_numpy(),
                       temp[['head_gaze_ry']].to_numpy(),
                       temp[['head_gaze_rz']].to_numpy(),
                       len(temp[['head_gaze_rz']].to_numpy()))
    li_temp.append(gamma_h)
    theta = getTheta(temp[['eye_gaze_rx', 'eye_gaze_ry', 'eye_gaze_rz']].to_numpy(),
                     temp[['head_gaze_rx', 'head_gaze_ry', 'head_gaze_rz']].to_numpy(),
                     len(temp[['eye_gaze_rx', 'eye_gaze_ry', 'eye_gaze_rz']].to_numpy()))
    li_temp.append(theta)
    phi = getPhi(temp[['eye_gaze_rx', 'eye_gaze_ry', 'eye_gaze_rz']].to_numpy(),
                 temp[['head_gaze_rx', 'head_gaze_ry', 'head_gaze_rz']].to_numpy(),
                 len(temp[['eye_gaze_rx', 'eye_gaze_ry', 'eye_gaze_rz']].to_numpy()))
    li_temp.append(phi)

    #Statistical Features (36)
    skeleton_head_x_std = temp['skeleton_head_x'].std()
    li_temp.append(skeleton_head_x_std)
    skeleton_head_y_std = temp['skeleton_head_y'].std()
    li_temp.append(skeleton_head_y_std)
    skeleton_head_z_std = temp['skeleton_head_z'].std()
    li_temp.append(skeleton_head_z_std)
    skeleton_sholder_center_x_std = temp['skeleton_sholder_center_x'].std()
    li_temp.append(skeleton_sholder_center_x_std)
    skeleton_sholder_center_y_std = temp['skeleton_sholder_center_y'].std()
    li_temp.append(skeleton_sholder_center_y_std)
    skeleton_sholder_center_z_std = temp['skeleton_sholder_center_z'].std()
    li_temp.append(skeleton_sholder_center_z_std)
    skeleton_sholder_left_x_std = temp['skeleton_sholder_left_x'].std()
    li_temp.append(skeleton_sholder_left_x_std)
    skeleton_sholder_left_y_std = temp['skeleton_sholder_left_y'].std()
    li_temp.append(skeleton_sholder_left_y_std)
    skeleton_sholder_left_z_std = temp['skeleton_sholder_left_z'].std()
    li_temp.append(skeleton_sholder_left_z_std)
    skeleton_sholder_right_x_std = temp['skeleton_sholder_right_x'].std()
    li_temp.append(skeleton_sholder_right_x_std)
    skeleton_sholder_right_y_std = temp['skeleton_sholder_right_y'].std()
    li_temp.append(skeleton_sholder_right_y_std)
    skeleton_sholder_right_z_std = temp['skeleton_sholder_right_z'].std()
    li_temp.append(skeleton_sholder_right_z_std)
    skeleton_elbow_left_x_std = temp['skeleton_elbow_left_x'].std()
    li_temp.append(skeleton_elbow_left_x_std)
    skeleton_elbow_left_y_std = temp['skeleton_elbow_left_y'].std()
    li_temp.append(skeleton_elbow_left_y_std)
    skeleton_elbow_left_z_std = temp['skeleton_elbow_left_z'].std()
    li_temp.append(skeleton_elbow_left_z_std)
    skeleton_elbow_right_x_std = temp['skeleton_elbow_right_x'].std()
    li_temp.append(skeleton_elbow_right_x_std)
    skeleton_elbow_right_y_std = temp['skeleton_elbow_right_y'].std()
    li_temp.append(skeleton_elbow_right_y_std)
    skeleton_elbow_right_z_std = temp['skeleton_elbow_right_z'].std()
    li_temp.append(skeleton_elbow_right_z_std)
    skeleton_hand_left_x_std = temp['skeleton_hand_left_x'].std()
    li_temp.append(skeleton_hand_left_x_std)
    skeleton_hand_left_y_std = temp['skeleton_hand_left_y'].std()
    li_temp.append(skeleton_hand_left_y_std)
    skeleton_hand_left_z_std = temp['skeleton_hand_left_z'].std()
    li_temp.append(skeleton_hand_left_z_std)
    skeleton_hand_right_x_std = temp['skeleton_hand_right_x'].std()
    li_temp.append(skeleton_hand_right_x_std)
    skeleton_hand_right_y_std = temp['skeleton_hand_right_y'].std()
    li_temp.append(skeleton_hand_right_y_std)
    skeleton_hand_right_z_std = temp['skeleton_hand_right_z'].std()
    li_temp.append(skeleton_hand_right_z_std)
    skeleton_wrist_left_x_std = temp['skeleton_wrist_left_x'].std()
    li_temp.append(skeleton_wrist_left_x_std)
    skeleton_wrist_left_y_std = temp['skeleton_wrist_left_y'].std()
    li_temp.append(skeleton_wrist_left_y_std)
    skeleton_wrist_left_z_std = temp['skeleton_wrist_left_z'].std()
    li_temp.append(skeleton_wrist_left_z_std)
    skeleton_wrist_right_x_std = temp['skeleton_wrist_right_x'].std()
    li_temp.append(skeleton_wrist_right_x_std)
    skeleton_wrist_right_y_std = temp['skeleton_wrist_right_y'].std()
    li_temp.append(skeleton_wrist_right_y_std)
    skeleton_wrist_right_z_std = temp['skeleton_wrist_right_z'].std()
    li_temp.append(skeleton_wrist_right_z_std)
    eye_gaze_rx_std = temp['eye_gaze_rx'].std()
    li_temp.append(eye_gaze_rx_std)
    eye_gaze_ry_std = temp['eye_gaze_ry'].std()
    li_temp.append(eye_gaze_ry_std)
    eye_gaze_rz_std = temp['eye_gaze_rz'].std()
    li_temp.append(eye_gaze_rz_std)
    head_gaze_rx_std = temp['head_gaze_rx'].std()
    li_temp.append(head_gaze_rx_std)
    head_gaze_ry_std = temp['head_gaze_ry'].std()
    li_temp.append(head_gaze_ry_std)
    head_gaze_rz_std = temp['head_gaze_rz'].std()
    li_temp.append(head_gaze_rz_std)

    #Demographic Features (3)
    li_temp.append(temp['participant_ageInMonths'].iloc[0])
    filters = [(temp.participant_ageInMonths >= 24) & (temp.participant_ageInMonths < 36),
               (temp.participant_ageInMonths >= 36) & (temp.participant_ageInMonths < 48),
               (temp.participant_ageInMonths >= 48) & (temp.participant_ageInMonths < 60),
               (temp.participant_ageInMonths >= 60) & (temp.participant_ageInMonths < 72),
               (temp.participant_ageInMonths >= 72) & (temp.participant_ageInMonths < 84)
              ]
    values = [1, 2, 3, 4, 5]
    age_group = np.select(filters, values)
    li_temp.append(age_group[0])
    temp['participant_gender'].replace({'male':0,
                                        'female':1},inplace = True)
    li_temp.append(temp['participant_gender'].iloc[0])

    #Task Category Features (2)
    temp["task_ability"].replace({'IM':0,
                                  'TT':1,
                                  'JA':2},inplace = True)
    li_temp.append(temp['task_ability'].iloc[0])
    li_temp.append(temp['task_difficultyLevel'].iloc[0])

    #Dependent Variables
    li_temp.append(temp['ados_preTest_communication'].iloc[0])
    li_temp.append(temp['ados_preTest_interaction'].iloc[0])
    li_temp.append(temp['ados_preTest_module'].iloc[0])
    li_temp.append(temp['ados_preTest_play'].iloc[0])
    li_temp.append(temp['ados_preTest_protocol'].iloc[0])
    li_temp.append(temp['ados_preTest_socialCommunicationQuestionnaire'].iloc[0])
    li_temp.append(temp['ados_preTest_stereotype'].iloc[0])
    li_temp.append(temp['ados_preTest_total'].iloc[0])
    temp["condition"].replace({'SHT':0,
                               'RET':1},inplace = True)
    li_temp.append(temp['condition'].iloc[0])
    filters_l = [(temp.ados_preTest_total >= 0) & (temp.ados_preTest_total < 6),
                 (temp.ados_preTest_total >= 7) & (temp.ados_preTest_total < 11),
                 (temp.ados_preTest_total >= 12) & (temp.ados_preTest_total < 24)
                ]
    values_l = [1, 2, 3]
    level = np.select(filters_l, values_l)
    li_temp.append(level[0])

    df_processed.loc[len(df_processed)] = li_temp

  print(i + 1)
  df_processed = df_processed.dropna()
  if df_processed.empty:
    continue
  li_processed.append(df_processed)

In [ ]:
df_final = pd.concat(li_processed, axis = 0, ignore_index = True)

In [ ]:
df_final.to_csv(url_save_loc, index=False)

In [ ]:
df_final.shape

In [ ]:
df_final.info()

In [ ]:
df_final.head()